In [1]:
# Importamos re para expresiones regulares
import os
import settings
from preprocessor import *

In [2]:
settings.initialize()

In [3]:
TEST_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.test.en')
CHECK_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.check.en')
TRAIN_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.train.en')

### APARTADO 1

Vamos a implementar un sistema que reciba una expresión como entrada (será una expresión formada solo por minúsculas y sin los signos de puntuación mencionados) y la salida será la misma expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de puntuación indicados.

Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.

In [6]:
from punctuator_basic import addPunctuationBasic

Vemos que la función anterior realiza la operación correctamente con el siguiente ejemplo.

In [7]:
addPunctuationBasic('esta es una frase de prueba')

'Esta es una frase de prueba.'

### APARTADO 2 

Antes de definir la función verifyPuntuation vamos a definir dos funciones auxiliares que nos servirán.
* padding(list1,list2): Devuelve las los listas de entrada de forma que ambas tengan la misma longitud, añadiendo el elemento string vacío ('') a la lista de menor longitud.

* tokenizer(text): tokeniza el texto de entrada.

Veamos su funcionamiento con el siguiente ejemplo:

In [8]:
list1, list2 = ['s','a'], ['1','2','3','4']
l1, l2 = padding(list1,list2)
print('Lista 1 con padding: ',l1,'\t Lista 2 con padding: ', l2)
print('Las longitudes son iguales: ', len(l1) == len(l2))

Lista 1 con padding:  ['s', 'a', '', ''] 	 Lista 2 con padding:  ['1', '2', '3', '4']
Las longitudes son iguales:  True


Veamos cómo funciona con el siguiente ejemplo:

In [9]:
text_example= "Sara said: Hello, what's your name?"
print(tokenizer(text_example))

['Sara', 'said', ':', 'Hello', ',', "what's", 'your', 'name', '?']


In [10]:
from evaluator import verifyPunctuation

Utilicemos el ejemplo del documento para ver si funciona correctamente la función. Además verificamos que intercambiar check y test devuelve el mismo número de elementos.

In [11]:
check_example = "Hello. What's your name?"
test_example = "Hello what's your, name?"
print('check vs test: ', verifyPunctuation(check_example,test_example))
print('test vs check: ', verifyPunctuation(test_example,check_example))

check vs test:  {('D', 1), ('S', 2), ('I', 4)}
test vs check:  {('S', 1), ('I', 1), ('D', 3)}


### APARTADO 3

Implementaremos una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, irá recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el algoritmo básico de puntuación (apartado 1: addPunctuationBasic() ) y a continuación comprobaría si el resultado es o no correcto usando la función verifyPunctuation() del apartado 2.

En primer lugar vamos a definir una función evaluate_example que calcula las métricas precision y recall dado una instancia (check,test). 

In [12]:
from evaluator import evaluate_example

Comprobamos el funcionamiento de la función con el siguiente ejemplo.

* check = "Hello. What's your name?"
* test = "hello what's your name"

In [13]:
check_example = "Hello. What's your name?"
test_example = "hello what's your name"
evaluate_example(addPunctuationBasic,check_example,test_example,print_info=True)

TEST LINE: 
  hello what's your name
MODEL PUNCTUATED LINE: 
  Hello what's your name.
VALIDATION LINE: 
  Hello. What's your name? 

Modificaciones necesarias:  {('S', 0), ('S', 1), ('I', 1), ('I', 4)}
Modificaciones hechas por el modelo:  {('S', 0), ('I', 4)}
Diferencias entre modelo y validación:  {('S', 1), ('I', 1), ('S', 4)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas: 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  4 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.25 



(0, 0.5, 0.25, 2, 1, 4)

Notemos que la oración puntuada es "Hello what's your name.", por lo que la función de validación va a considerar como correcto el haber insertado en el token 4, lo que se observa como ('I', 4) tanto en las modificaciones necesarias como las hechas por el modelo. Al no tener información sobre el caracter, esto podría dar lugar a un falso positivo, que es corregido correctamente con el error de sustitución de signo visto en la función.

Definamos ahora la función evaluate(...) que raliza el proceso anterior para distintos corpus y calcula métricas gobales, es decir, la que pide el propio apartado 3.

In [14]:
from evaluator import evaluate

Evaluamos la función addPunctuationBasic:

In [15]:
evaluate(addPunctuationBasic, check_file_path=CHECK_RAW_PATH, test_file_path=TEST_RAW_PATH)

MÉTRICAS
precision global:  0.9481749791028141
recall global:  0.4281984334203655
F1 global:  0.5899664102286271
precision media:  0.9474342928660826
recall medio:  0.5873733513179556
F1 medio:  0.7251692521379949
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382


{'precision_global': 0.9481749791028141,
 'recall_global': 0.4281984334203655,
 'F1_global': 0.5899664102286271,
 'precision_mean': 0.9474342928660826,
 'recall_mean': 0.5873733513179556,
 'F1_mean': 0.7251692521379949,
 'score': 0.263384786538729}

Vemos que la precisión es alta, lo que indica que lo que tiene que hacer el modelo (poner la primera letra en mayúscula y un punto al final), lo hace bien. Sin embargo, el recall es relativamente bajo ya que esos cambios no son suficientes para puntuar correctamente las oraciones, cosa que también se deduce del bajo rendimiento.

Aunque no se pide, definimos la función evaluate_example_from_corpus que permite ver la información resultante de evaluate_example para un elemento en concreto del corpus dado el número de línea corpus_line. De esta forma podemos explorar y verificar el correcto funcionamiento de las funciones definidas.

In [16]:
from evaluator import evaluate_example_from_corpus

In [17]:
evaluate_example_from_corpus(addPunctuationBasic,check_file_path=CHECK_RAW_PATH,test_file_path=TEST_RAW_PATH ,corpus_line=50)

TEST LINE: 
  and what do i mean by that
MODEL PUNCTUATED LINE: 
  And what do i mean by that.
VALIDATION LINE: 
  And what do I mean by that? 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('S', 3)}
Modificaciones hechas por el modelo:  {('S', 0), ('I', 7)}
Diferencias entre modelo y validación:  {('S', 7), ('S', 3)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas: 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.3333333333333333 



### APARTADO 4

Utilizando el corpus de entrenamiento contenido en PunctuationTask.train.en construimos un modelo de lenguaje inspirado en la idea de 4-gramas.

Definimos una función auxiliar ngrams(text,N) que dado el string text devuelve la lista de N-gramas.

In [18]:
from preprocessor import ngrams

Veamos un ejemplo:

In [19]:
text_example = 'Sara said: Hello! nice to meet you!'
for n in range(2,5):
    print(str(n)+'-gramas de la oración: ', text_example)
    print('\t', ngrams(text_example,n))

2-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said'), ('said', ':'), (':', 'Hello'), ('Hello', '!'), ('!', 'nice'), ('nice', 'to'), ('to', 'meet'), ('meet', 'you'), ('you', '!')]
3-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':'), ('said', ':', 'Hello'), (':', 'Hello', '!'), ('Hello', '!', 'nice'), ('!', 'nice', 'to'), ('nice', 'to', 'meet'), ('to', 'meet', 'you'), ('meet', 'you', '!')]
4-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':', 'Hello'), ('said', ':', 'Hello', '!'), (':', 'Hello', '!', 'nice'), ('Hello', '!', 'nice', 'to'), ('!', 'nice', 'to', 'meet'), ('nice', 'to', 'meet', 'you'), ('to', 'meet', 'you', '!')]


Creamos la clase ModelNgram, que puede generalizarse fácilmente y que instanciaremos con N=4 para definir el modelo propuesto del apartado 4.

In [20]:
from model_ngram import ModelNgram

Intanciamos el modelo y lo entrenamos

In [21]:
model4gram = ModelNgram(N=4)
model4gram.entrena(train_file_path=TRAIN_RAW_PATH)

Consultamos el diccionario de conteo para comprobar la distribución de signos dada la terna ('by','the','way')

In [22]:
terna = ('by','the','way')
for i in model4gram.counts_dict:
    print('Ocurrencias con el signo ',i,' para la terna ',terna, ':',model4gram.counts_dict[i].get(terna,0))

Ocurrencias con el signo  <mayus>  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  <minus>  para la terna  ('by', 'the', 'way') : 25
Ocurrencias con el signo  .  para la terna  ('by', 'the', 'way') : 36
Ocurrencias con el signo  ,  para la terna  ('by', 'the', 'way') : 200
Ocurrencias con el signo  ;  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  :  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  ?  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  !  para la terna  ('by', 'the', 'way') : 0


Se observa que la mayor ocurrencia se da para la coma ',' por lo que la función predice() del modelo debe devolver dicho caracter.

In [23]:
print('La operación más probable dada la terna ',terna, ' es ',model4gram.predice(terna))

La operación más probable dada la terna  ('by', 'the', 'way')  es  ,


Igual que definimos el modelo de forma genérica para Ngramas, vamos a definir la función genérica addPunctuationNgram donde la función requerida por el apartado addPunctuation4gram es addPunctuationNgram usando un modelo basado en 4 gramas.

In [24]:
from punctuator_ngram import addPunctuationNgram

Podemos definir ahora la función de puntuación addPunctuation4gram simplemente como la ejecución de addPunctuationNgram
verificando que el modelo usado usa 4gramas.

In [25]:
def addPunctuation4gram(model,example,add_basic_punct = False):
    N = model.N
    assert N == 4, 'The model is based in {} -grams and it should be 4-grams.'.format(str(N))
    return addPunctuationNgram(model,example,add_basic_punct = add_basic_punct)


In [26]:
text_example = "and we also are eating meat that comes from some of these same places"
print('Frase sin puntuar:\n \t',text_example)
print('Puntuación de la frase con modelo 4 gramas:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=False))
print('Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=True))

Frase sin puntuar:
 	 and we also are eating meat that comes from some of these same places
Puntuación de la frase con modelo 4 gramas:
 	 and we also are eating meat, that comes from some of these same places
Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:
 	 And we also are eating meat, that comes from some of these same places.


Exploremos algunos ejemplos usando puntuación básica o no en el modelo de 4gramas

In [27]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i = 0

l = 70
print('='*l)
print('MODELO 4GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=False,corpus_line = i)
print('='*l)
print('='*l)
print('4GRAMS + PUNTUACION BÁSICA')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram, test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,add_punct_basic=True,corpus_line = i)
print('='*l)

MODELO 4GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  it can be a very complicated thing, the ocean
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('I', 9)}
Modificaciones hechas por el modelo:  {('I', 7)}
Diferencias entre modelo y validación:  {('I', 10), ('S', 0)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  1
n_correctas (Núm. de modificaciones correctas: 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  1.0
recall (n_correctas/n_necesarias):  0.3333333333333333 

4GRAMS + PUNTUACION BÁSICA
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 7), ('I', 9)}

### APARTADO 5

Evaluemos el modelo usando la misma función evaluate() anterior. Vamos a comparar el rendimiento usando también la puntuación básica y sin ella.

In [28]:
print('MODELO 4GRAMS')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH)
print()

MODELO 4GRAMS
MÉTRICAS
precision global:  0.3060887512899897
recall global:  0.04665135738777564
F1 global:  0.08096303979909374
precision media:  0.14062073775442538
recall medio:  0.049548668299233246
F1 medio:  0.07327751014820442
rendimiento:  0.0
número de instancias en el corpus:  14382



Podemos ver que este modelo tal cual es muy pobre. Usando la función evaluate_example_from_corpus(), podemos ver que nunca pone la mayúscula inicial como era de esperar y rara vez un punto al final. Además las predicciones parecen bastante pobres y esto se puede deber a la falta de variedad en en las tuplas del corpus de entrenamiento. El corpus debería ser más grande y variado. 

Veamos la evaluación añadiendo la puntuación básica y lo comparamos con lo obtenido en addPunctuationBasic.

In [29]:
print('4GRAMS + PUNTUACION BÁSICA')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,add_punct_basic=True)
print()
print('PUNTUACIÓN BÁSICA')
evaluate(addPunctuationBasic,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,)
print()

4GRAMS + PUNTUACION BÁSICA
MÉTRICAS
precision global:  0.7800087656823536
recall global:  0.4478750511183114
F1 global:  0.5690220215019384
precision media:  0.8431887161340575
recall medio:  0.5991128246565686
F1 medio:  0.700498694835623
rendimiento:  0.2369628702544848
número de instancias en el corpus:  14382

PUNTUACIÓN BÁSICA
MÉTRICAS
precision global:  0.9481749791028141
recall global:  0.4281984334203655
F1 global:  0.5899664102286271
precision media:  0.9474342928660826
recall medio:  0.5873733513179556
F1 medio:  0.7251692521379949
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382



Podemos ver que en general la precisión es más baja en el modelo de 4gramas pero el recall ligeramente superior, pero no de forma sustancial. Además los F1 son menores en el modelo de puntuación básica que en el de 4 gramas así como el rendimiento.

En general podemos concluir que el modelo de puntuación básica es mejor que el basado en 4gramas cuando se añade la puntuación línea a línea por sorprendente que parezca.

Probemos ahora a añadir la puntuación al corpus completo y evaluarlo.

NOTA: La siguiente celda esta comentada para evitar ejecutarla sin querer ya que tarda mucho en ejecutarse.

In [30]:
#print('MODELO 4GRAMS')
#evaluate(addPunctuation4gram,model=model4gram,all_file=True)
#print()

Ya que hemos implementado un modelo genérico, vamos a comparar con modelos basados en 3gramas y 5gramas para ver si existe mejoría o no.

In [31]:
model3gram = ModelNgram(N=3)
model5gram = ModelNgram(N=5)
model3gram.entrena(train_file_path=TRAIN_RAW_PATH)
model5gram.entrena(train_file_path=TRAIN_RAW_PATH)

In [32]:
print('3GRAMAS')
evaluate(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True)

3GRAMAS
MÉTRICAS
precision global:  0.7267446209080256
recall global:  0.45847620245997045
F1 global:  0.5622498481005332
precision media:  0.8039312953739148
recall medio:  0.6054353508863484
F1 medio:  0.6907051861838088
rendimiento:  0.2259769155889306
número de instancias en el corpus:  14382

5GRAMAS
MÉTRICAS
precision global:  0.8714429298322759
recall global:  0.4363930919500456
F1 global:  0.5815586484447053
precision media:  0.9020709308151896
recall medio:  0.5925814372753306
F1 medio:  0.7152840354304869
rendimiento:  0.2533027395355305
número de instancias en el corpus:  14382



In [ ]:
print('5GRAMAS')
evaluate(addPunctuationNgram, model = model5gram, test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,add_punct_basic=True)
print()

Observamos que en cuanto a precision, el modelo basado en 3gramas es ligeramente inferior al de 4gramas y este a su vez inferior al de 5 gramas, todos por debajo de la precisión del puntuador básico. En cuanto a recall el orden es inverso: el modelo basado en 3 gramas presenta mayor recall que el de 4 y este a su vez que el de 5.
En cuanto al F1, los mayores valores los encontramos para el modelo de 5gramas y de puntuación básica. Es obvio que cuanto mayor sea N, menos probable es la probabilidad de que se de una N-tupla concreta por lo que el modelo basado en N-gramas con N grande coincidirá eventualmente con el de puntuación básica si se considera el parámetro add_punct_basic.

Para acabar este apartado podemos explorar algunas predicciones de los modelos de 3 y 5 gramas:

In [679]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i= 0

l = 70 
print('='*l)
print('MODELO 3GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)
print('='*l)
print('MODELO 5GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model5gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)

MODELO 3GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing. The ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('I', 9), ('I', 7)}
Modificaciones hechas por el modelo:  {('S', 0), ('I', 9), ('S', 7), ('I', 7)}
Diferencias entre modelo y validación:  {('S', 7), ('S', 8)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  4
n_correctas (Núm. de modificaciones correctas: 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  2
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.6666666666666666 

MODELO 5GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('S', 0), ('

### APARTADO 6

In [4]:
TEST_PREP_PATH  = prepare_file(TEST_RAW_PATH,'test.prepared.txt')
CHECK_PREP_PATH = prepare_file(CHECK_RAW_PATH,'check.prepared.txt')
TRAIN_PREP_PATH = prepare_file(TRAIN_RAW_PATH,'train.prepared.txt')

In [5]:
from preprocessor import train_dev_test_split

In [6]:
TRAIN_TRAIN_PREP_PATH, TRAIN_DEV_PREP_PATH, TRAIN_TEST_PREP_PATH = train_dev_test_split(TRAIN_PREP_PATH, train_split = 0.7, dev_split = 0.15)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
!python .punctuator2tf2/data.py settings.DATA_PREPARED_DIR

In [ ]:
!python .punctuator2tf2/main.py punctuator 10 0.02

In [ ]:
!python ./punctutator2tf2/punctuator.py ./Model_punctuator_h256_lr0.02.pcl TEST_RAW_PATH .predicted/data.model_output.test.txt 